In [55]:
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn

In [56]:
class Inception(nn.Block):
    def __init__(self,n1_1,n2_1,n2_3,n3_1,n3_5,n4_1,**kwargs):
        super(Inception,self).__init__(**kwargs)
        self.p1_con_1 = nn.Conv2D(n1_1,kernel_size=1,activation="relu")
        
        self.p2_con_1 = nn.Conv2D(n2_1,kernel_size=1,activation="relu")
        self.p2_con_3 = nn.Conv2D(n2_3,kernel_size=3,padding=1,activation="relu")
        
        self.p3_con_1 = nn.Conv2D(n3_1,kernel_size=1,activation="relu")
        self.p3_con_5  = nn.Conv2D(n3_5,kernel_size=5,padding=2,activation="relu")
        
        self.p4_maxpool = nn.MaxPool2D(pool_size=3,padding=1,strides=1)
        self.p4_con_1 = nn.Conv2D(n4_1,kernel_size=1,activation="relu")
        
    def forward(self,x):
        p1 = self.p1_con_1(x)
        p2 = self.p2_con_3(self.p2_con_1(x))
        p3 = self.p3_con_5(self.p3_con_1(x))
        p4 = self.p4_con_1(self.p4_maxpool(x))
        
        return nd.concat(p1,p2,p3,p4,dim=1)
        

In [57]:
incp = Inception(64, 96, 128, 16, 32, 32)
incp.initialize()

x = nd.random.uniform(shape=(32,3,64,64))
incp(x).shape

(32, 256, 64, 64)

In [58]:
a = Inception(128,64,192,64,96,64)
a.initialize()
x = nd.random.uniform(shape=(256,3,28,28))
a(x).shape

(256, 480, 28, 28)

In [70]:
class googleNet(nn.Block):
    def __init__(self,num_classes,verbose=False,**kwargs):
        super(googleNet,self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            b1 = nn.Sequential()
            b1.add(
                nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation="relu"),
                nn.MaxPool2D(pool_size=3,strides=2)
            )
            b2 = nn.Sequential()
            b2.add(
                nn.Conv2D(64,kernel_size=1,activation="relu"),
                nn.Conv2D(192,kernel_size=3,padding=1,activation="relu"),
                nn.MaxPool2D(pool_size=3,strides=2)
            )
            b3 = nn.Sequential()
            b3.add(
                Inception(64,96,128,16,32,32),
                Inception(128,128,192,32,96,64),
                nn.MaxPool2D(pool_size=3,strides=2)
                  )
            b4 = nn.Sequential()
            b4.add(
                Inception(192,96,208,16,48,64),
                Inception(160,112,224,24,64,64),
                Inception(128,128,256,24,64,64),
                Inception(112,144,288,32,64,64),
                Inception(256,160,320,32,128,128),
                nn.MaxPool2D(pool_size=3,strides=2)
            )
            b5= nn.Sequential()
            b5.add(
                Inception(256,160,320,32,128,128),
                Inception(384,192,384,48,128,128),
                nn.AvgPool2D(pool_size=2)
            )
            b6 = nn.Sequential()
            b6.add(
                nn.Flatten(),
                nn.Dense(num_classes,activation="relu")
            )
            #chain block
            self.b= nn.Sequential()
            self.b.add(b1,b2,b3,b4,b5,b6)
            
    def forward(self,x):
        out = x 
        for i,value in enumerate(self.b):
            out = value(out)
            if self.verbose:
                 print('Block %d output: %s'%(i+1, out.shape))
        return out

In [71]:
net = googleNet(10,verbose=True)
net.initialize()

x = nd.random.uniform(shape=(4, 3, 96, 96))
y = net(x)

Block 1 output: (4, 64, 23, 23)
Block 2 output: (4, 192, 11, 11)
Block 3 output: (4, 480, 5, 5)
Block 4 output: (4, 832, 2, 2)
Block 5 output: (4, 1024, 1, 1)
Block 6 output: (4, 10)


In [75]:
import sys
sys.path.append('./')
import utils1
from mxnet import gluon
from mxnet import init

train_data, test_data = utils1.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils1.try_gpu()
net = googleNet(10)
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.01})



In [77]:
utils1.train(train_data, test_data, net, loss,trainer, ctx, num_epochs=2)

Start training on  gpu(0)
Epoch 0. Loss: 2.297, Train acc 0.23, Test acc 0.26, Time 116.6 sec
Epoch 1. Loss: 2.181, Train acc 0.30, Test acc 0.41, Time 115.3 sec
